<a href="https://colab.research.google.com/github/ProfessorDong/Deep-Learning-Course-Examples/blob/master/RNN_Examples/CharacterGeneratorRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Natural Language Processing (Character Generation) with Recurrent Neural Network


We will generate charater using a RNN.  We will work with a dataset of Shakespeare's writing.  Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

- The model is character-based.
- The model is trained on small batches of text (100 characters each), and is able to generate a longer sequence of text with coherent structure.

*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*


In [1]:
# %tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

###Download the Shakespeare Dataset

Here, we use an extract from a Shadespheare play for training.  We can use our own text paragraph data and use it for network training.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# If you load your own data file, use the following lines:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

1130496/1115394 [==============================] - 0s 0us/step


###Read Contents of File
Let's look at the contents of the file.

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


###Encode
Encode each unique character as a different integer.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# Chect how a part of the text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


Make a function that can convert the numeric values back to text.


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Create Training Examples

Given a character, or a sequence of characters, what is the most probable next character? This is the task we are training the model to perform.

Our task is to feed the model a sequence and have it return the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

In [9]:
# Create a stream of characters from our text data
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Use the batch method to turn this stream of characters into batches of desired length
seq_length = 100  # length of sequence for a training example
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
# Use these sequences of length 101 and split them into input and output.
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [10]:
for x, y in dataset.take(3):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


EXAMPLE

INPUT
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us k

OUTPUT
ow Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki


Make training batches.

In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Build the Model
We use an embedding layer, a LSTM layer, and one dense layer that contains a node for each unique character in our training data.  The dense layer will give us a probability distribution over all nodes.

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    # tf.keras.layers.LSTM(rnn_units,
    #                     return_sequences=True,
    #                     stateful=True,
    #                     recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),                        
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


###Create a Loss Function
Our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

We will create the loss function.

Before we do that, let's have a look at a sample input and the output from the untrained model. 

In [14]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [16]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-5.41907270e-03 -2.28211354e-03 -3.66019271e-03 ... -6.11666078e-03
   -5.28287329e-03  6.02365006e-03]
  [ 4.64769453e-03 -2.95270677e-03  3.22469673e-03 ... -1.16034634e-02
   -1.58145872e-03 -6.18119910e-03]
  [-1.56538188e-02  8.58875737e-03 -6.33563427e-03 ... -4.85565327e-03
    4.50722314e-03  7.62340939e-03]
  ...
  [-5.36419079e-03  2.53915763e-03  8.83678533e-03 ...  7.12119276e-03
    4.34004748e-03  9.30017326e-03]
  [ 6.16940530e-03 -6.59750821e-03  8.65066238e-03 ... -7.96890445e-03
    6.16318686e-03  1.11605627e-02]
  [-1.31823914e-02 -8.30625836e-03  6.00653992e-04 ...  4.95325867e-03
    1.28209172e-02  1.20109813e-02]]

 [[-4.39409603e-04 -5.17007243e-03 -1.88792066e-03 ... -2.48594698e-03
   -1.43894702e-02 -9.43676569e-03]
  [ 6.01488864e-04 -1.25777686e-03 -6.03386294e-03 ... -1.58663839e-02
   -1.15654543e-02  1.46452319e-02]
  [-1.82442386e-02 -6.21139910e-03 -3.99835967e-03 ... -1.82705163e-03
    4.74584027e-04  1.75715946e-02]
  ...
  [-1.783

In [17]:
# Let's examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00541907 -0.00228211 -0.00366019 ... -0.00611666 -0.00528287
   0.00602365]
 [ 0.00464769 -0.00295271  0.0032247  ... -0.01160346 -0.00158146
  -0.0061812 ]
 [-0.01565382  0.00858876 -0.00633563 ... -0.00485565  0.00450722
   0.00762341]
 ...
 [-0.00536419  0.00253916  0.00883679 ...  0.00712119  0.00434005
   0.00930017]
 [ 0.00616941 -0.00659751  0.00865066 ... -0.0079689   0.00616319
   0.01116056]
 [-0.01318239 -0.00830626  0.00060065 ...  0.00495326  0.01282092
   0.01201098]], shape=(100, 65), dtype=float32)


In [18]:
# and finally we look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-5.4190727e-03 -2.2821135e-03 -3.6601927e-03 -8.7351790e-03
 -3.5764945e-03 -2.2440463e-05  1.2761970e-03  3.6944025e-03
  1.6294751e-02 -8.7446384e-03 -8.0211824e-03  6.6128308e-03
 -2.1985148e-03  9.1846511e-03 -5.2645379e-03 -1.6030746e-03
  6.3992976e-03 -4.6629515e-03  1.4977583e-02  1.3883000e-02
  8.5811578e-03 -2.8973041e-04  9.3631912e-03  8.8136923e-03
 -1.1557942e-02  5.7224142e-03 -6.2229964e-03  7.6710409e-03
 -7.8579551e-03 -9.9105593e-03  3.0639644e-03 -1.3814080e-02
 -3.2387162e-03  6.8541844e-03  2.2471272e-03  1.8290582e-04
 -3.4013114e-04 -3.1481474e-03 -1.2262067e-02 -3.5310080e-03
 -6.2473852e-04  5.7526343e-03  3.2707502e-03 -5.5208704e-03
  3.1069147e-03 -2.2983563e-03  9.8220855e-03  2.2051334e-03
  1.6061060e-02 -1.1400103e-02 -5.1306286e-03  2.7428120e-03
  1.7796766e-03  4.6124007e-03 -1.1046658e-02  2.4535670e-03
 -8.6697321e-03  8.5174544e-03 -5.6210672e-03  4.0255575e-03
  7.7150133e-04 -2.7500172e-03 -6.1166608e-03 -5.2828733e-03
  6.023650

In [19]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'c?J-.F3Lc!TOZFBl!qzMF-,iuX-KC ?tzQnKHiUu,;EKLvoC!MRTw,vu :Tig-ToLJwHS\nQVH3xVPjdIxigVjw Ndm,\nzj-sw!$Q'

So, we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were. 

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compile the Model
We can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next. 


In [21]:
model.compile(optimizer='adam', loss=loss)

###Create Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Train the Model
**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**

In [23]:
history = model.fit(data, epochs=30, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 26s 130ms/step - loss: 2.6833
Epoch 2/30
172/172 [==============================] - 24s 130ms/step - loss: 1.9779
Epoch 3/30
172/172 [==============================] - 24s 130ms/step - loss: 1.7098
Epoch 4/30
172/172 [==============================] - 24s 130ms/step - loss: 1.5580
Epoch 5/30
172/172 [==============================] - 24s 131ms/step - loss: 1.4664
Epoch 6/30
172/172 [==============================] - 24s 131ms/step - loss: 1.4067
Epoch 7/30
172/172 [==============================] - 24s 130ms/step - loss: 1.3602
Epoch 8/30
172/172 [==============================] - 24s 130ms/step - loss: 1.3207
Epoch 9/30
172/172 [==============================] - 24s 130ms/step - loss: 1.2863
Epoch 10/30
172/172 [==============================] - 24s 130ms/step - loss: 1.2532
Epoch 11/30
172/172 [==============================] - 24s 130ms/step - loss: 1.2214
Epoch 12/30
172/172 [==============================] - 24s 130ms/step - lo

###Load the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one piece of text to the model and have it make a prediction.

In [24]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model finishes training, we can find the **lastest checkpoint** that stores the models weights using the following line.

In [25]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [ ]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

###Generate Text
Generate some text using any starting string we choose.

Generating text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.  Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

![alt text](https://www.tensorflow.org/text/tutorials/images/text_generation_sampling.png)


In [26]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [27]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: This is a test
This is a testimation of my imprison.

KING RICHARD II:
Mine issue even for death.
Her sister Katharina!

LUCIO:
But, ere I did so: by shape, faint, tread!'

JULIET:
Montague, Meneniug, at Francis perasence
So to or one silp send the beggary hangments of sons out
Marcius petition goes to love a fair
due is to bed.
And kind use me with the dishonour, though they kiss the drowsy
as my performal to thy means and men exhabitable dang!
us gone?

Second Servant:
O, then, dost thou did forget that robby hour?

DUKE OF YORK:
He was too good for me; let's seize upon him,
Make your best end, by small stopp'd denial;
For sorry as I find the other lives.

Romany,
Cannot confess:
We have received your partisans, and e,
Sin, or how but we?

CAPULET:
Go, get you gone both.

DUKE VINCENTIO:
A Citizen:
I say usurp and s


##Sources

1. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.
2. “Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
3. “Understanding LSTM Networks.” Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.